# Load in Metadata and Edits

In [ ]:
library("plyr")
source("scripts/REDIT_LLR.R")

In [ ]:
setwd("/oasis/tscc/scratch/m7huang/COVID_editing/")

In [ ]:
# Remove conflicting samples
ignore_samples <- c("COVSUBJ_0137_1_N_HA", "COVSUBJ_0146_1_N_HA", "COVSUBJ_0376_1_N_HA",
                    "COVSUBJ_0461_1_N_HA", "COVSUBJ_0558_1_N_HA", "COVSUBJ_0619_1_N_HA")

metadata <- read.table("rdata/Updated_Cornell_Metadata_08072020_KF_20210908.txt",
                       sep="\t", stringsAsFactors=FALSE, header = TRUE)
metadata$Tumor_Sample_Barcode <- metadata$SampleID
metadata <- subset(metadata, !(SampleID %in% ignore_samples))
table(metadata$Type)

In [ ]:
# Read in all RNA edits and combine
edits <- do.call(rbind, lapply(list.files("."), fread, stringsAsFactors=FALSE))
edits$VarID <- paste0(edits$Chromosome, ":", edits$Start_Position)
edits$Tumor_Sample_Barcode <- gsub("_filtered.human", "", edits$Tumor_Sample_Barcode)
write.table(edits, file="rnaedits/merged_rnaedits_raw.tsv", sep="\t", quote=FALSE, row.names=FALSE)
sitesCount <- arrange(data.frame(table(edits$VarID)), -Freq)

# keep edits present in at least 20 - 5% samples
edits_keep <- subset(sitesCount, Freq >= 35)$Var1
edits_filt <- subset(edits, VarID %in% edits_keep)

edits_filt_metadata <- merge(edits_filt, metadata, by="Tumor_Sample_Barcode")
write.table(edits_filt, file="rnaedits/merged_rnaedits_filtered.tsv", sep="\t", quote=FALSE, row.names=FALSE)
save(edits_filt, file="rdata/merged_rnaedits_filtered_5percent.RData")

# Expand Edits to Run REDIT_LLR

In [ ]:
expandSite <- function(site, samples, maf, metadata){
  site_maf <- cbind(Tumor_Sample_Barcode=samples, site)
  site_maf <- merge(site_maf, maf[,c("Tumor_Sample_Barcode", "Chromosome", "Start_Position", "End_Position", "t_alt_count", "t_ref_count")],
                    by=names(site_maf), all.x=TRUE)
  site_maf$t_alt_count[is.na(site_maf$t_alt_count)] <- 0
  mean_ref_count <- round(mean(site_maf$t_ref_count, na.rm=TRUE), 0)
  site_maf$t_ref_count[is.na(site_maf$t_ref_count)] <- mean_ref_count
  site_maf <- merge(site_maf, metadata, by="Tumor_Sample_Barcode")
  unique(site_maf)
}

expandMaf <- function(maf, metadata){
    samples <- unique(maf$Tumor_Sample_Barcode)
    sites <- unique(maf[,c("Chromosome", "Start_Position", "End_Position")])
    subset_maf <- maf[, c("Tumor_Sample_Barcode", "Chromosome", "Start_Position", "End_Position", "t_alt_count", "t_ref_count")]
    expanded_maf <- do.call(rbind, lapply(1:nrow(sites), function(i) expandSite(sites[i,], samples, subset_maf, metadata)))
}

In [ ]:
# Create Expanded Edits
expanded_edits <- expandMaf(edits_filt, metadata)
expanded_edits$VarID <- paste0(expanded_edits$Chromosome, ":", expanded_edits$Start_Position)
expanded_edits
save(expanded_edits, file="rdata/expanded_edits.RData")

expanded_edits$VarID <- paste0(expanded_edits$Chromosome, ":", expanded_edits$Start_Position)

# Determine Fraction of Samples Edited

In [ ]:
doRedits <- function(maf, site, column){
  maf_site <- subset(maf, VarID == site)
  the_data <- rbind(maf_site$t_alt_count, maf_site$t_ref_count)
  the_groups <- as.character(maf_site[[column]])
  REDIT_LLR(data=the_data, groups=the_groups)
}

pAdjDiffEdits <- function(diffEditPValues, sites_by_gene, edit_sites){
  p_vals <- lapply(diffEditPValues, `[[`, "p.value")
  names(p_vals) <- edit_sites
  p_adj <- p.adjust(p_vals)
  p_sig <- p_adj[p_adj < 0.05]
  p_sig_df <- data.frame(VarID=names(p_sig), p.adj=p_sig)
  p_sig_df <- join(p_sig_df, sites_by_gene)
  p_sig_df <- unique(p_sig_df)
  #write.table(p_sig_df, file=sprintf("%s_sig_diffedit.tsv", comparison), sep="\t", row.names=FALSE, quote=FALSE)
  p_sig_df
}

fractionOfSamplesEdited <- function(p_adj_results, maf, metadata, column, condition1, condition2){
    # Identify unique mafs (only sig p-value)
    sig_edit_sites <- unique(maf$VarID)
    sig_edit_sites_x <- lapply(sig_edit_sites, function(i) subset(maf, VarID == i))

    # Calculate fraction of samples edited at each site
    yes_infection <- lapply(sig_edit_sites_x, function(i) nrow(i[i[[column]] == condition1,])/nrow(metadata[metadata[[column]] == condition1,]))
    no_infection <- lapply(sig_edit_sites_x, function(i) nrow(i[i[[column]] == condition2, ])/nrow(metadata[metadata[[column]] == condition2, ]))
                     
    # Format into dataframe, combine with p-values
    res_df <- data.frame(unlist(sig_edit_sites), unlist(yes_infection), unlist(no_infection))
    colnames(res_df) <- c("VarID", condition1, condition2)
    res_df <- merge(res_df, p_adj_results, by = "VarID")
}

In [ ]:
# For: COVID-19 Vs. OVI
covid_ovi_samples <- subset(metadata, Type != "None")$Tumor_Sample_Barcode

# For: COVID-19 High Vs. Low
high_low_samples <- subset(metadata, Type %in% c("High", "Low"))$Tumor_Sample_Barcode

# For: COVID-19 Vs. None
not_ovi_samples <- subset(metadata, Type != "OtherViralInfection")$Tumor_Sample_Barcode

# For: OVI Vs. None
not_covid_samples <- subset(metadata, Type %in% c("OtherViralInfection", "None"))$Tumor_Sample_Barcode

In [ ]:
metadata_covid_ovi <- subset(metadata, Tumor_Sample_Barcode %in% covid_ovi_samples)
metadata_covid_ovi$COVIDOrOVI <- ifelse(metadata_covid_ovi$Type == "OtherViralInfection", yes="OtherViralInfection", no="COVID")

metadata_high_low <- subset(metadata, Tumor_Sample_Barcode %in% high_low_samples)
metadata_high_low$HighOrLow <- ifelse(metadata_high_low$Type == "High", yes="High", no="Low")

metadata_not_ovi <- subset(metadata, Tumor_Sample_Barcode %in% not_ovi_samples)
metadata_not_ovi$ViralInfection <- ifelse(metadata_not_ovi$Type == "None", yes="NoInfection", no="YesInfection")

metadata_not_covid <- subset(metadata, Tumor_Sample_Barcode %in% not_covid_samples)
metadata_not_covid$OVIorNone <- ifelse(metadata_not_covid$Type == "OtherViralInfection", yes="OtherViralInfection", no="None")

In [ ]:
# For: COVID-19  Vs. OVI

# Subset Expanded Edits for High and Low samples only
expanded_edits_covid_ovi <- subset(expanded_edits, Tumor_Sample_Barcode %in% covid_ovi_samples)
expanded_edits_covid_ovi$COVIDOrOVI <- ifelse(expanded_edits_covid_ovi$Type == "OtherViralInfection", yes="OtherViralInfection", no="COVID")

expanded_edits_covid_ovi <- expanded_edits_covid_ovi[,c("VarID", "COVIDOrOVI", "Type", "t_ref_count", "t_alt_count")]

# Identify unique editing sites
editSites_covid_ovi <- list()
editSites_covid_ovi <- unique(expanded_edits_covid_ovi$VarID)
save(editSites_covid_ovi, file = "rnaedits/editSites_covid_ovi.RData")

# Call REDIT_LLR for each unique editing site
diffEdits_covid_ovi <- list()
diffEdits_covid_ovi <- lapply(editSites_covid_ovi, function(i) doRedits(maf=expanded_edits_covid_ovi, site=i, column="COVIDOrOVI"))
                               
save(diffEdits_covid_ovi, file = "rnaedits/diffEdits_covid_ovi.RData")

In [ ]:
expanded_edits_high_low <- subset(expanded_edits, Tumor_Sample_Barcode %in% high_low_samples)
expanded_edits_high_low$HighOrLow <- ifelse(expanded_edits_high_low$Type == "High", yes="High", no="Low")

expanded_edits_high_low <- expanded_edits_high_low[,c("VarID", "HighOrLow", "Type", "t_ref_count", "t_alt_count")]

# Identify unique editing sites
editSites_high_low <- list()
editSites_high_low <- unique(expanded_edits_high_low$VarID)
save(editSites_high_low, file = "rnaedits/editSites_high_low.RData")

# Call REDIT_LLR for each unique editing site
diffEdits_high_low <- list()
diffEdits_high_low <- lapply(editSites_high_low, function(i) doRedits(maf=expanded_edits_high_low, site=i, column="HighOrLow"))
                               
save(diffEdits_high_low, file = "rnaedits/diffEdits_high_low.RData")


In [ ]:
expanded_edits_not_ovi <- subset(expanded_edits, Tumor_Sample_Barcode %in% not_ovi_samples)
expanded_edits_not_ovi$COVIDOrNone <- ifelse(expanded_edits_not_ovi$Type == "None", yes="None", no="COVID")

expanded_edits_not_ovi <- expanded_edits_not_ovi[,c("VarID", "COVIDOrNone", "Type", "t_ref_count", "t_alt_count")]

# Identify unique editing sites
editSites_not_ovi <- list()
editSites_not_ovi <- unique(expanded_edits_not_ovi$VarID)
save(editSites_not_ovi, file = "rnaedits/editSites_not_ovi.RData")

# Call REDIT_LLR for each unique editing site
diffEdits_not_ovi <- list()
diffEdits_not_ovi <- lapply(editSites_not_ovi, function(i) doRedits(maf=expanded_edits_not_ovi, site=i, column="COVIDOrNone"))
                               
save(diffEdits_not_ovi, file = "rnaedits/diffEdits_not_ovi.RData")

In [ ]:
expanded_edits_not_covid <- subset(expanded_edits, Tumor_Sample_Barcode %in% not_covid_samples)
expanded_edits_not_covid$OVIorNone <- ifelse(expanded_edits_not_covid$Type == "None", yes="None", no="OtherViralInfection")

expanded_edits_not_covid <- expanded_edits_not_covid[,c("VarID", "OVIorNone", "Type", "t_ref_count", "t_alt_count")]

# Identify unique editing sites
editSites_not_covid <- list()
editSites_not_covid <- unique(expanded_edits_not_covid$VarID)
save(editSites_not_covid, file = "rnaedits/editSites_not_covid.RData")

# Call REDIT_LLR for each unique editing site
diffEdits_not_covid <- list()
diffEdits_not_covid <- lapply(editSites_not_covid, function(i) doRedits(maf=expanded_edits_not_covid, site=i, column="OVIorNone"))
                               
save(diffEdits_not_covid, file = "rnaedits/diffEdits_not_covid.RData")

In [ ]:
edits_filt_covid_ovi <- subset(edits_filt_metadata, Tumor_Sample_Barcode %in% covid_ovi_samples)
edits_filt_covid_ovi$COVIDOrOVI <- ifelse(edits_filt_covid_ovi$Type == "OtherViralInfection", yes="OtherViralInfection", no="COVID")
edits_filt_covid_ovi <- edits_filt_covid_ovi[, c("Tumor_Sample_Barcode", "Type", "COVIDOrOVI", "Hugo_Symbol", "VarID", "Variant_Classification")]

edit_sites_by_gene_covid_ovi <- edits_filt_covid_ovi[,c("Hugo_Symbol", "VarID", "Variant_Classification")]

edit_sites_by_gene_covid_ovi <- subset(edit_sites_by_gene_covid_ovi, VarID %in% editSites_covid_ovi)
p_adj_covid_ovi <- pAdjDiffEdits(diffEdits_covid_ovi, edit_sites_by_gene_covid_ovi, editSites_covid_ovi)

# Merge to only include p.value < 0.05 sites
edits_filt_covid_ovi <- merge(edits_filt_covid_ovi, p_adj_covid_ovi, by = c("VarID", "Variant_Classification", "Hugo_Symbol"))

p_adj_covid_ovifractions <- fractionOfSamplesEdited(p_adj_covid_ovi, edits_filt_covid_ovi, metadata_covid_ovi, "COVIDOrOVI", "OtherViralInfection", "COVID")

In [ ]:
edits_filt_high_low <- subset(edits_filt_metadata, Tumor_Sample_Barcode %in% high_low_samples)
edits_filt_high_low$HighOrLow <- ifelse(edits_filt_high_low$Type == "High", yes="High", no="Low")
edits_filt_high_low <- edits_filt_high_low[, c("Tumor_Sample_Barcode", "Type", "HighOrLow", "Hugo_Symbol", "VarID", "Variant_Classification")]

edit_sites_by_gene_high_low <- edits_filt_high_low[,c("Hugo_Symbol", "VarID", "Variant_Classification")]

edit_sites_by_gene_high_low <- subset(edit_sites_by_gene_high_low, VarID %in% editSites_high_low)
p_adj_high_low <- pAdjDiffEdits(diffEdits_high_low, edit_sites_by_gene_high_low, editSites_high_low)

# Merge to only include p.value < 0.05 sites
edits_filt_high_low <- merge(edits_filt_high_low, p_adj_high_low, by = c("VarID", "Variant_Classification", "Hugo_Symbol"))

p_adj_high_lowfractions <- fractionOfSamplesEdited(p_adj_high_low, edits_filt_high_low, metadata_high_low, "HighOrLow", "High", "Low")

In [ ]:
edits_filt_not_ovi <- subset(edits_filt_metadata, Tumor_Sample_Barcode %in% not_ovi_samples)
edits_filt_not_ovi$ViralInfection <- ifelse(edits_filt_not_ovi$Type == "None", yes="NoInfection", no="YesInfection")
edits_filt_not_ovi <- edits_filt_not_ovi[, c("Tumor_Sample_Barcode", "Type", "ViralInfection", "Hugo_Symbol", "VarID", "Variant_Classification")]

edit_sites_by_gene_not_ovi <- edits_filt_not_ovi[,c("Hugo_Symbol", "VarID", "Variant_Classification")]
p_adj_not_ovi <- pAdjDiffEdits(diffEdits_not_ovi, edit_sites_by_gene_not_ovi, editSites_not_ovi)

# Merge to only include p.value < 0.05 sites
edits_filt_not_ovi <- merge(edits_filt_not_ovi, p_adj_not_ovi, by = c("VarID", "Variant_Classification", "Hugo_Symbol"))

p_adj_not_ovi_fractions <- fractionOfSamplesEdited(p_adj_not_ovi, edits_filt_not_ovi, metadata_not_ovi, "ViralInfection", "YesInfection", "NoInfection")


In [ ]:
edits_filt_not_covid <- subset(edits_filt_metadata, Tumor_Sample_Barcode %in% not_covid_samples)
edits_filt_not_covid$OVIorNone <- ifelse(edits_filt_not_covid$Type == "OtherViralInfection", yes="OtherViralInfection", no="None")
edits_filt_not_covid <- edits_filt_not_covid[, c("Tumor_Sample_Barcode", "Type", "OVIorNone", "Hugo_Symbol", "VarID", "Variant_Classification")]

edit_sites_by_gene_not_covid <- edits_filt_not_covid[,c("Hugo_Symbol", "VarID", "Variant_Classification")]

edit_sites_by_gene_not_covid <- subset(edit_sites_by_gene_not_covid, VarID %in% editSites_not_covid)
p_adj_not_covid <- pAdjDiffEdits(diffEdits_not_covid, edit_sites_by_gene_not_covid, editSites_not_covid)

# Merge to only include p.value < 0.05 sites
edits_filt_not_covid <- merge(edits_filt_not_covid, p_adj_not_covid, by = c("VarID", "Variant_Classification", "Hugo_Symbol"))

p_adj_not_covid_fractions <- fractionOfSamplesEdited(p_adj_not_covid, edits_filt_not_covid, metadata_not_covid, "OVIorNone", "OtherViralInfection", "None")

In [ ]:
write.table(p_adj_covid_ovifractions, file="rnaedits/differentialediting_padj_covidvsovi.tsv", sep="\t", quote=FALSE, row.names=FALSE)
write.table(p_adj_high_lowfractions, file="rnaedits/differentialediting_padj_highvslow.tsv", sep="\t", quote=FALSE, row.names=FALSE)
write.table(p_adj_not_ovi_fractions, file="rnaedits/differentialediting_padj_covidvsnone.tsv", sep="\t", quote=FALSE, row.names=FALSE)
write.table(p_adj_not_covid_fractions, file="rnaedits/differentialediting_padj_ovivsnone.tsv", sep="\t", quote=FALSE, row.names=FALSE)

In [1]:
sessionInfo()

R version 4.2.2 (2022-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /home/m7huang/miniconda3/envs/single_cell_notebook/lib/libopenblasp-r0.3.21.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

loaded via a namespace (and not attached):
 [1] fansi_1.0.3     crayon_1.5.2    digest_0.6.31   utf8_1.2.2     
 [5] IRdisplay_1.1   repr_1.1.5      lifecycle_1.0.3 jsonlite_1.8.4 
 [9] evaluate_0.19   pillar_1.8.1    rlang_1.1.1     cli_3.6.0      
[13] uuid_1.1-0      vctrs_0.6.2     IRkernel_1.3.1  tools_4.